## Importing Used Libraries

In [2]:
from collections import defaultdict
import heapq

## Graph Class

In [3]:
class Graph:
  def __init__(self):
    self.graph = defaultdict(list) # Create empty dictionary that stores lists
    self.heuristic = {}

  def add_edge(self, u_node, v_node, weight): # Add edge using weighted adjacency list
    self.graph[u_node] += [(v_node, weight)]
    self.graph[v_node] += [(u_node, weight)]

  def add_heuristic(self, node, value): # Add heuristic function value to a node
    self.heuristic[node] = value

## Shortest Path Algorithms

In [4]:
# Dijkstra Algorithm
def dijkstra(graph, start_node, goal):
    distance = {start_node: 0}
    heap_queue = [(distance[start_node], start_node)]
    parent = {start_node: start_node} 

    for node in graph.graph:
        if node != start_node:
            distance[node] = float('inf')
            heapq.heappush(heap_queue, (distance[node], node))
    
    while len(heap_queue) > 0:
        current_node = heapq.heappop(heap_queue)[1]

        for (neighbour_node, weight) in graph.graph[current_node]:
            if neighbour_node in [arr[1] for arr in heap_queue] \
                and distance[current_node] != float('inf') \
                and weight + distance[current_node] < distance[neighbour_node]:
                
                heap_queue = [node for node in heap_queue if node[1] != neighbour_node]
                
                distance[neighbour_node] = weight + distance[current_node]
                parent[neighbour_node] = current_node

                heapq.heappush(heap_queue, (distance[neighbour_node], neighbour_node))
    
    path = []
    while parent[goal] != goal:
        path.append(goal)
        goal = parent[goal]
    
    path.append(start_node)
    path.reverse()
    return path, distance

In [5]:
# Bellman-Ford Algorithm
def bellman_ford(graph, start_node, goal):
    distance = {node: float('inf') for node in graph.graph}
    distance[start_node] = 0

    parent = {start_node: start_node}

    for _ in range(len(graph.graph) - 1):
        for current_node in graph.graph:
            for neighbour_node, weight in graph.graph[current_node]:
                if distance[current_node] != float('inf') and distance[current_node] + weight < distance[neighbour_node]:
                    parent[neighbour_node] = current_node
                    distance[neighbour_node] = distance[current_node] + weight
                
    for current_node in graph.graph:
        for neighbour_node, weight in graph.graph[current_node]:
            if distance[current_node] != float('inf') and distance[current_node] + weight < distance[neighbour_node]:
                return None, None
            
    path = []
    while parent[goal] != goal:
        path.append(goal)
        goal = parent[goal]
    
    path.append(start_node)
    path.reverse()

    return path, distance

In [6]:
# A* Search Algorithm
def a_star(graph, start_node, goal):
    open_set = set([start_node]) # To store nodes to look next
    closed_set = set() # To store nodes visited
    heap_queue = [(0, start_node)]

    g = {start_node: 0} # Store cumulative distance of a node
    parent = {start_node: start_node} # Store parent of a node

    while open_set:
      current_node = heapq.heappop(heap_queue)[1] # Pop node with smallest f(x)

      if current_node == goal:
        path = []
        distance = g[current_node]

        while parent[current_node] != current_node: # Retrace path using parent dictionary
          path.append(current_node)
          current_node = parent[current_node]

        path.append(start_node)
        path.reverse()
        return (path, distance)
      
      for (neighbour_node, weight) in graph.graph[current_node]: # Iterate all neighbours of current node
        if neighbour_node not in open_set and neighbour_node not in closed_set:
          open_set.add(neighbour_node)
          g[neighbour_node] = g[current_node] + weight # Set cumulative distance of neighbour node
          parent[neighbour_node] = current_node        # Set parent of neighbour node to current node
          f = graph.heuristic[neighbour_node] + g[neighbour_node] # Calcualate f function
          heapq.heappush(heap_queue, (f, neighbour_node)) # Push to heap queue
        
        elif g[neighbour_node] > g[current_node] + weight: # If neighbour has shorter distance through this path
          g[neighbour_node] = g[current_node] + weight
          parent[neighbour_node] = current_node
          heapq.heappush(heap_queue, (graph.heuristic[neighbour_node] + g[neighbour_node], neighbour_node))

          if neighbour_node in closed_set: # Readd neighbour node to open set
            closed_set.remove(neighbour_node)
            open_set.add(neighbour_node)
      
      open_set.remove(current_node)
      closed_set.add(current_node)

In [7]:
# Floyd Warshall Algorithm
def floyd_warshall(graph, start_node, goal):
    distance = {node_u: {node_v: float('inf') for node_v in graph.graph} for node_u in graph.graph}

    previous = {node_u: {node_v: None for node_v in graph.graph} for node_u in graph.graph}
    for node in graph.graph:
        distance[node][node] = 0
    
    for node, neighbors in graph.graph.items():
        for neighbor, weight in neighbors:
            distance[node][neighbor] = weight
            previous[node][neighbor] = node

    for k in graph.graph:
        for i in graph.graph:
            for j in graph.graph:
                if distance[i][k] + distance[k][j] < distance[i][j]:
                    distance[i][j] = distance[i][k] + distance[k][j]
                    previous[i][j] = previous[k][j]

    path = []
    while goal != start_node:
        if goal is None:
            return None
        path.append(goal)
        goal = previous[start_node][goal]
    path.append(start_node)
    path.reverse()

    return path, distance

In [8]:
# Johnson's Algorithm
def johnson(graph, start_node, goal):
    graph.graph['S'] = [(node, 0) for node in graph.graph]

    _, distance = bellman_ford(graph, 'S', 'V1')

    if distance is None:
        return None, None
    
    for node in graph.graph:
        graph.graph[node] = [(neighbor, weight + distance[node] - distance[neighbor]) for neighbor, weight in graph.graph[node]]
    
    graph.graph.pop('S')

    return dijkstra(graph, start_node, goal)

## Testing Each Algorithm

In [9]:
# Initializing and adding edges to graph
graph = Graph()

graph.add_edge('V1','V2', weight=2)
graph.add_edge('V1','V3', weight=8)
graph.add_edge('V1','V4', weight=1)

graph.add_edge('V2','V3', weight=6)
graph.add_edge('V2','V5', weight=1)

graph.add_edge('V3','V4', weight=7)
graph.add_edge('V3','V5', weight=5)
graph.add_edge('V3','V6', weight=1)
graph.add_edge('V3','V7', weight=2)

graph.add_edge('V4','V7', weight=9)

graph.add_edge('V5','V6', weight=3)
graph.add_edge('V5','V8', weight=2)
graph.add_edge('V5','V9', weight=9)

graph.add_edge('V6','V7', weight=4)
graph.add_edge('V6','V9', weight=6)

graph.add_edge('V7','V9', weight=3)
graph.add_edge('V7','V10', weight=1)

graph.add_edge('V8','V9', weight=7)
graph.add_edge('V8','V11', weight=9)

graph.add_edge('V9','V10', weight=1)
graph.add_edge('V9','V11', weight=2)

graph.add_edge('V10','V11', weight=4)


# Adding heuristic values to graph
graph.add_heuristic('V1', 10)
graph.add_heuristic('V2', 9)
graph.add_heuristic('V3', 8)
graph.add_heuristic('V4', 9)
graph.add_heuristic('V5', 7)
graph.add_heuristic('V6', 6)
graph.add_heuristic('V7', 7)
graph.add_heuristic('V8', 5)
graph.add_heuristic('V9', 3)
graph.add_heuristic('V10', 5)
graph.add_heuristic('V11', 0)

#### Running Dijkstra Algorithm

In [10]:
(lintasan, jarak) = dijkstra(graph, 'V1', 'V11')

print('Shortest Path    :', lintasan)
print('Shortest Distance:', jarak['V11'])

Shortest Path    : ['V1', 'V2', 'V5', 'V6', 'V3', 'V7', 'V10', 'V9', 'V11']
Shortest Distance: 13


#### Running Bellman-Ford Algorithm

In [11]:
(lintasan, jarak) = bellman_ford(graph, 'V1', 'V11')

print('Shortest Path    :', lintasan)
print('Shortest Distance:', jarak['V11'])

Shortest Path    : ['V1', 'V2', 'V5', 'V6', 'V3', 'V7', 'V10', 'V9', 'V11']
Shortest Distance: 13


#### Running A* Search Algorithm

In [12]:
(lintasan, jarak) = a_star(graph, 'V1', 'V11')

print('Shortest Path    :', lintasan)
print('Shortest Distance:', jarak)

Shortest Path    : ['V1', 'V2', 'V5', 'V8', 'V11']
Shortest Distance: 14


#### Running Floyd Warshall

In [13]:
(lintasan, jarak) = floyd_warshall(graph, 'V1', 'V11')

print('Shortest Path    :', lintasan)
print('Shortest Distance:', jarak['V1']['V11'])


Shortest Path    : ['V1', 'V2', 'V5', 'V6', 'V3', 'V7', 'V10', 'V9', 'V11']
Shortest Distance: 13


#### Running Johnson's Algorithm

In [14]:
(lintasan, jarak) = johnson(graph, 'V1', 'V11')

print('Shortest Path    :', lintasan)
print('Shortest Distance:', jarak['V11'])


Shortest Path    : ['V1', 'V2', 'V5', 'V6', 'V3', 'V7', 'V10', 'V9', 'V11']
Shortest Distance: 13
